# Monitor Vertex AI Pipeline Execution

This notebook helps monitor the status of your pipeline execution.

In [ ]:
from google.cloud import aiplatform
from google.auth import default
import time

# Get default credentials and project
credentials, project_id = default()

# Initialize Vertex AI
aiplatform.init(
    project=project_id,
    location='us-central1',
    credentials=credentials
)

In [ ]:
def monitor_pipeline(pipeline_job_id):
    """Monitor pipeline execution status and details."""
    job = aiplatform.PipelineJob(pipeline_job_id)
    
    print(f"Pipeline Name: {job.display_name}")
    print(f"State: {job.state}")
    print(f"Create Time: {job.create_time}")
    
    if job.state == 'PIPELINE_STATE_RUNNING':
        print("\nRunning Tasks:")
        for task in job.task_details:
            if task.state == 'RUNNING':
                print(f"- {task.task_name}: {task.state}")
    
    print("\nCompleted Tasks:")
    for task in job.task_details:
        if task.state == 'SUCCEEDED':
            print(f"- {task.task_name}: {task.state}")
            
    if job.state == 'PIPELINE_STATE_FAILED':
        print("\nFailed Tasks:")
        for task in job.task_details:
            if task.state == 'FAILED':
                print(f"- {task.task_name}: {task.error}")

def continuous_monitoring(pipeline_job_id, interval=60):
    """Continuously monitor pipeline until completion."""
    job = aiplatform.PipelineJob(pipeline_job_id)
    
    while job.state in ['PIPELINE_STATE_RUNNING', 'PIPELINE_STATE_PENDING']:
        print("\n" + "="*50)
        print(f"Status Check at {time.strftime('%Y-%m-%d %H:%M:%S')}")
        monitor_pipeline(pipeline_job_id)
        time.sleep(interval)
        job = aiplatform.PipelineJob(pipeline_job_id)
    
    print("\nPipeline Execution Completed")
    print(f"Final State: {job.state}")
    monitor_pipeline(pipeline_job_id)

In [ ]:
# Replace with your pipeline job ID
pipeline_job_id = "YOUR_PIPELINE_JOB_ID"  # You'll get this from the main execution notebook

# Monitor once
print("Current Pipeline Status:")
monitor_pipeline(pipeline_job_id)

In [ ]:
# Continuous monitoring (updates every 60 seconds)
continuous_monitoring(pipeline_job_id, interval=60)